In [2]:
import json
import pandas as pd
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing import sequence
from keras.utils import np_utils

import os
import shutil
import urllib3
import zipfile
import pandas as pd
import json
import ast

In [3]:
f = open("[라벨]경상도_학습데이터_1/DKCI20000001.json", encoding="UTF-8")
raw_data = json.loads(f.read())

In [4]:
# 사투리, 표준어 사전 생성 함수
def make_dict(raw_data):

    dialect_word = []
    standard_word = [] 
    
    
    for i in range(len(raw_data["utterance"])):
        eojeol = raw_data["utterance"][i]["eojeolList"]
            
        for k in range(len(eojeol)):
            if eojeol[k]["isDialect"] == True:
                if k not in dialect_word:
                    dialect_word.append(eojeol[k]['eojeol'])
                    standard_word.append(eojeol[k]['standard'])

    word_dictionary = pd.DataFrame([dialect_word, standard_word])
    word_dictionary = word_dictionary.transpose()
    word_dictionary.columns = ['dialect', 'standard']
    dialect_dictionary = word_dictionary['dialect']
    standard_dictionary = word_dictionary['standard']
    dialect_dict = dialect_dictionary.to_dict()
    standard_dict = standard_dictionary.to_dict()

    index_to_dialect = {v:k for k,v in dialect_dict.items()}
    index_to_standard = {v:k for k,v in standard_dict.items()}

    dialect_to_index = {v:k for k,v in index_to_dialect.items()}
    standard_to_index = {v:k for k,v in index_to_standard.items()}

    print(len(index_to_dialect))
    print(len(dialect_to_index))

    print(len(index_to_standard))
    print(len(standard_to_index))
    

    return dialect_to_index,standard_to_index,index_to_dialect,index_to_standard
   

In [5]:
index_to_dialect,index_to_standard,dialect_to_index,standard_to_index = make_dict(raw_data)

236
236
223
223


In [6]:
# 사투리 표준어 쌍 생성 함수
def make_pair(raw_data):
    filter = re.compile('[^가-힣+]')

    dialect_form = []
    standard_form = []

    for i in range(len(raw_data["utterance"])):
        sentence = raw_data["utterance"][i]
        
        for j in range(len(sentence["eojeolList"])):
            #print(sentence["eojeolList"][0]['eojeol'])
            if sentence["eojeolList"][j]['isDialect'] == True:
                try : 

                    dialect = sentence["eojeolList"][j]["eojeol"]
                    standard = sentence["eojeolList"][j]["standard"]
                    
                    dialect = filter.sub('', dialect)
                    standard = filter.sub('', standard)
                    
                    dialect = dialect_to_index[dialect]
                    standard = standard_to_index[standard]

                    dialect_form.append(dialect)
                    standard_form.append(standard)
                except:
                    pass

    result = pd.DataFrame(data = {"dialect_words":dialect_form, "standard_words": standard_form})


    return result
               

In [7]:
pairs = make_pair(raw_data)

In [8]:
# 사투리 사전에 존재하는지 확인하는 함수

def exist_in_dialect(x):

    if x in dialect_to_index:
        return True
    else:
        return False


In [9]:
exist_in_dialect("자가격리")

False

In [10]:
xtrain = np_utils.to_categorical(pairs["dialect_words"])
ytrain = np_utils.to_categorical(pairs["standard_words"])
len(xtrain[0])

416

In [11]:
xtrain.shape

(398, 416)

In [12]:
translate_model = Sequential()
translate_model.add(Dense(256, input_dim = 416, activation="relu"))
translate_model.add(Dense(416, activation="softmax"))

In [13]:
translate_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
translate_model.fit(xtrain, ytrain, batch_size=100, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 6.0181 - accuracy: 0.0377
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 5.9630 - accuracy: 0.2588
Epoch 3/100
4/4 [==============================] - 0s 7ms/step - loss: 5.9087 - accuracy: 0.3844
Epoch 4/100
4/4 [==============================] - 0s 6ms/step - loss: 5.8511 - accuracy: 0.4724
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 5.7877 - accuracy: 0.4899
Epoch 6/100
4/4 [==============================] - 0s 8ms/step - loss: 5.7174 - accuracy: 0.5176
Epoch 7/100
4/4 [==============================] - 0s 9ms/step - loss: 5.6391 - accuracy: 0.5025
Epoch 8/100
4/4 [==============================] - 0s 8ms/step - loss: 5.5511 - accuracy: 0.4950
Epoch 9/100
4/4 [==============================] - 0s 6ms/step - loss: 5.4523 - accuracy: 0.4950
Epoch 10/100
4/4 [==============================] - 0s 6ms/step - loss: 5.3420 - accuracy: 0.4824
Epoch 11/100
4/4 [===========

In [17]:
# 사투리 단어 번역 함수
def translate(word):
    index = dialect_to_index[word]
    one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
    pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
    res = index_to_standard[pred_index]


    # for i in range(len(sentence)):
    #     if exist_in_dialect(sentence[i]) == True:
    #         print(dialect_to_index[sentence[i]])
    #         index = dialect_to_index[sentence[i]]
    #         one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
    #         pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
    #         res = res + " " + index_to_standard[pred_index]

    #     else:
    #         res = res + " " + sentence[i]

    return res


In [16]:
translate("자가격리 이주 했다 아이가")

NameError: name 'sentence' is not defined

In [24]:
exist_in_dialect("인덱스")

False

In [25]:
pos_dict = {}

In [26]:
def make_eojeol(standard_to_index, dialect_to_index):
    eojeol_list=[]
    st_list = []
    di_list = []
    
    for k in standard_to_index:
        st_list.append(k)
        
    for v in dialect_to_index:
        di_list.append(v)
        
    for i, j in zip(st_list, di_list):
        
        eojeol_list.append(i)
        
        if i == j:
            eojeol_list.append(j + "0")
        else:
            eojeol_list.append(j)   
            
    eojeol_df = pd.DataFrame(eojeol_list, columns=None)
    eojeol_dict = eojeol_df.to_dict()
    eojeol_dict = eojeol_dict[0]
    eojeol_dict = {v:k for k,v in eojeol_dict.items()}
    print(eojeol_dict)

    return eojeol_dict

In [27]:
eojeol_dict = make_eojeol()


In [28]:
def eojeol_dependence():
    eojeol_d=[]
    pos_d=[]
    #dic=[]
    re_eojeol_dict = {v:k for k,v in eojeol_dict.items()} #{인덱스:문자}
        
    for i in range(len(dep)): #dep(딕셔너리)길이만큼 반복 23
        if dep[i]['text'] in eojeol_dict:
            eojeol_d.append(eojeol_dict.values())
        for j in range(len(dep[i]['mod'])):
            mod=dep[i]['mod']
            if len(mod[j]) > 0:
                pos=re_eojeol_dict.get(mod[j])#결과값:문자
                #mod가 가리키는 어절의 품사를 확인하는 방법..
                if dep[i]['label'] in pos_dict:
                    pos_d.append(pos_dict.values())
                else:
                    return None
                
    result = pd.DataFrame(data = {"word_dic":eojeol_d, "mod_dic":pos_d})
    return result



In [198]:
def predict_synonym(sentence):
    res = ""
    sentence_sep = sentence.split()
    print(sentence_sep)
    for i in range(len(sentence_sep)):
        print(i)
        if exist_in_dialect(sentence_sep[i]) == True: #사투리 사전에 있으면
            if sentence_sep[i] in eojeol_dict.keys(): #해당 단어가 전체 사전에 있으면

                pos, _ = get_mods(sentence, i) #연관된 단어의 품사 받아옴
                pos_idx = pos_dict[pos] #품사의 인덱스
                word = sentence_sep[i] 
                word0_idx = eojeol_dict[word+"0"] #단어0의 인덱스
                word_idx = eojeol_dict[word] #단어의 인덱스

                #one_hot = np_utils.to_categorical(pos_idx,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                percentage = tmp_model.predict(np.array([one_hot])) # 동의어 처리 모델
                res_idx = max(percentage[word0_idx], percentage[word_idx]) # 둘 중 더 큰 확률값
                
                one_hot = np_utils.to_categorical(res_idx,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                pred_index = np.argmax(translate_model.predict(np.array([one_hot]))) # 번역 모델                

                print("번역완료")

                res = res + " " + index_to_standard[pred_index]
            else: #해당 단어가 전체 사전에 없으면 -> 동의어가 없으므로 바로 번역 모델에 넣음
                word = sentence_sep[i] #단어의 인덱스
                word0_idx = eojeol_dict[word+"0"] #단어0의 인덱스
                one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
        else:
            res = res + " " + index_to_standard[pred_index]
            
    
    return res

In [161]:
def get_mods(sentence, num):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    analysisCode = "dparse"

    text = sentence
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))
    dict_with_mods = tmp["return_object"]["sentence"][0]["dependency"]

    #print(dict_with_mods[num]["mod"][0])
    id=int(dict_with_mods[num]["mod"][0])
    pos = dict_with_mods[id]["label"]
    
    return pos, dict_with_mods[id]["text"]